In [49]:
#!pip install -q ipython-autotime
%load_ext autotime
%load_ext autoreload
%timeit
%autoreload 2

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 1.11 s (started: 2023-07-25 13:11:26 +03:00)


In [50]:
import pandas as pd
import numpy as np
import torch 
from torch import nn
from Utils import *
from preprocess import *
from models import ANN, CNN1D, BILSTM
from torch.utils.data import Dataset, DataLoader



time: 125 ms (started: 2023-07-25 13:11:27 +03:00)


In [58]:
loaded_data = np.load("cleaned-dataset\data_npy.npy")
loaded_data.shape

(44266, 101)

time: 140 ms (started: 2023-07-25 13:14:31 +03:00)


In [51]:
loaded_data = np.load("cleaned-dataset\data_npy.npy")
X_train, X_test, y_train, y_test = split_data(loaded_data, ratio=0.70, seed=42)

#TODO #* SPLIT TO TRAIN, VAL, TEST to improve the model

training_dataset = TextDataset(X_train, y_train)
testing_dataset = TextDataset(X_test, y_test)
train_loader = DataLoader(dataset=training_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=training_dataset, batch_size=128, shuffle=True)

time: 125 ms (started: 2023-07-25 13:11:27 +03:00)


In [52]:
def acc_func(y_true, y_pred):
    y_pred = torch.round(y_pred)
    return (y_true == y_pred).sum() / len(y_true)

time: 110 ms (started: 2023-07-25 13:11:27 +03:00)


In [53]:
x, y = next(iter(train_loader))
print(x.shape), print(y.shape)

torch.Size([128, 100])
torch.Size([128])


(None, None)

time: 110 ms (started: 2023-07-25 13:11:27 +03:00)


In [54]:
from models import CNN1D
model = CNN1D()

# model(x.unsqueeze(-1)).shape

time: 110 ms (started: 2023-07-25 13:11:27 +03:00)


In [55]:
#* GLOABL VARIABLES
EPOCHS = 150
LEARNING_RATE = 3e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DEVICE = torch.device("cpu")

#* LSTM 
INPUT_SIZE = train_loader.dataset[0][0].shape[0]
HIDDEN_STATE = 64
NUM_LAYERS = 4
NUM_CLASSES = 1 #* binary classification

model = CNN1D().to(DEVICE)
# model = BILSTM(INPUT_SIZE, HIDDEN_STATE, NUM_LAYERS, NUM_CLASSES, bidirection=True).to(DEVICE)


loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)





time: 110 ms (started: 2023-07-25 13:11:27 +03:00)


In [56]:
# torch.save(ann.state_dict(), "model.pth")
# ann_loaded = ANN().to(DEVICE)
# ann_loaded.load_state_dict(torch.load("model.pth"))

time: 157 ms (started: 2023-07-25 13:11:28 +03:00)


In [57]:
history = Training(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    DEVICE,
    loss,
    optimizer,
    print_every=1,
    load_saved_model=False,
)

Epoch 0 | Train Loss: 0.52517
Epoch 0 | Train Acc: 81.89%
Epoch 0 | Test Loss: 0.44652
Epoch 0 | Test Acc: 91.42%
Test loss decreased to 0.44652 saving new best model
--------------------------------------------------
Epoch 1 | Train Loss: 0.41914
Epoch 1 | Train Acc: 86.07%
Epoch 1 | Test Loss: 0.27621
Epoch 1 | Test Acc: 88.68%
Test loss decreased from 0.44652 to 0.27621 saving new best model
--------------------------------------------------
Epoch 2 | Train Loss: 0.27006
Epoch 2 | Train Acc: 86.27%
Epoch 2 | Test Loss: 0.20563
Epoch 2 | Test Acc: 91.43%
Test loss decreased from 0.27621 to 0.20563 saving new best model
--------------------------------------------------
Epoch 3 | Train Loss: 0.24845
Epoch 3 | Train Acc: 87.60%
Epoch 3 | Test Loss: 0.19056
Epoch 3 | Test Acc: 92.12%
Test loss decreased from 0.20563 to 0.19056 saving new best model
--------------------------------------------------
Epoch 4 | Train Loss: 0.23718
Epoch 4 | Train Acc: 88.15%
Epoch 4 | Test Loss: 0.18234
Ep

In [ ]:
history = pd.DataFrame(history)
history.dropna(inplace=True)
history
# import matplotlib.pyplot as plt


# fig = plt.figure(figsize=(10, 7))
# plt.plot(history["accuracy"], label="train_acc")
# # plt.plot(history["val_accuracy"][:-1], label="val_acc")

# #* change the axix limit
# plt.ylim(0, 1)

ValueError: All arrays must be of the same length

time: 203 ms (started: 2023-07-25 13:01:27 +03:00)


In [ ]:
# loaded_ann = ANN().to(DEVICE)
# loaded_ann.load_state_dict(torch.load("ANN_best_model.pth"))

time: 125 ms (started: 2023-07-25 12:51:53 +03:00)


In [ ]:
path = "C:\\Users\\DELL\\OneDrive\\Desktop\\AI projects\\Fake-News-Detection\\word2vec_model\\word2vec.model"
word2vec_model = Load_word2vec(path)




Word2Vec model loaded successfully!
time: 875 ms (started: 2023-07-25 13:01:50 +03:00)


In [ ]:
from preprocess import *
sample = "The billionaire changed his profile picture to the new logo and added X.com to his Twitter bio."
#0.01
predict(sample, word2vec_model, model, DEVICE, prepare_for=model._get_name())
# sample = preprocess_text(sample)
# sample = vectorize_text(sample, word2vec_model)
# sample = torch.tensor(sample).float().to(DEVICE).unsqueeze(0).unsqueeze(1)
# print(sample.shape)
# model.eval()
# model(sample)
# print(predict(sample, word2vec_model, loaded_ann, DEVICE))

'Fake, with probability: 28.77%'

time: 1.75 s (started: 2023-07-25 13:01:52 +03:00)


In [ ]:
# Importing metrics
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Prediction
y_pred = ann(torch.tensor(X_test).float().to(DEVICE)).to("cpu").detach().numpy().reshape(-1)


# Accuracy score
print("Model accuracy is ",accuracy_score(y_pred,y_test))

confusion_matrix = confusion_matrix(y_pred=y_pred,y_true=y_test)

fig,ax = plt.subplots(figsize=(6,6))
sns.heatmap(confusion_matrix,annot=True,fmt="0.1f",linewidths=1.5)
plt.show()

NameError: name 'ann' is not defined

time: 250 ms (started: 2023-07-25 12:51:56 +03:00)
